In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, TwoSlopeNorm
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
data_licences = pd.read_parquet("data/data_licences/data_licences.parquet")
gdf_dep = gpd.read_file("departements.geojson")

In [ ]:
# On retire les départements non cartographiables et on crée une copie
data_licences_clean = data_licences.dropna(subset=["code_dep"]).copy()

# Conversion en string
data_licences_clean["code_dep"] = data_licences_clean["code_dep"].astype(str)

In [ ]:
# Fonction d'aggrégation
def aggregate_by_year(df, year):
    df_year = df[df["Année"] == year]
    return df_year.groupby("code_dep")["Licences annuelles"].sum().reset_index()

In [ ]:
def plot_licences(annee, sport='all', title=None):
    # filtrage sport
    if sport != 'all':
        df_filtered = data_licences_clean[data_licences_clean["code_sport"] == sport]
    else:
        df_filtered = data_licences_clean.copy()

    # agrégation
    df_agg = df_filtered[df_filtered["annee"] == annee].groupby("code_dep")["licences_annuelles"].sum().reset_index()

    # fusion avec la géométrie
    gdf_plot = gdf_dep.merge(df_agg, left_on="code", right_on="code_dep", how="left")

    # figure
    fig, ax = plt.subplots(figsize=(10, 12))

    gdf_plot.plot(
        column="licences_annuelles",
        ax=ax,
        legend=True,
        cmap="OrRd",
        edgecolor="grey",
        linewidth=0.5,
        missing_kwds={
            "color": "lightgrey",
            "edgecolor": "grey",
            "hatch": "//",
            "label": "Données manquantes"
        }
    )

    ax.set_title(title if title else f"Nombre de licenciés ({'tous les sports' if sport=='all' else sport}) par département ({annee})", fontsize=16)
    ax.set_axis_off()
    plt.show()


In [ ]:
plot_licences(2021, sport='ESC')

In [ ]:
def plot_evolution(annee1, annee2, sport='all', title=None):

    # filtrage sport
    if sport != 'all':
        df_filtered = data_licences_clean[data_licences_clean["code_sport"] == sport]
    else:
        df_filtered = data_licences_clean.copy()

    # agrégation pour les deux années
    df1 = df_filtered[df_filtered["annee"] == annee1].groupby("code_dep")["licences_annuelles"].sum().reset_index()
    df2 = df_filtered[df_filtered["annee"] == annee2].groupby("code_dep")["licences_annuelles"].sum().reset_index()

    # merge pour calcul du taux
    df_merge = df1.merge(df2, on="code_dep", suffixes=(f"_{annee1}", f"_{annee2}"))

    # taux de croissance : (B - A) / A
    df_merge["taux"] = (df_merge[f"licences_annuelles_{annee2}"] - df_merge[f"licences_annuelles_{annee1}"]) / df_merge[f"licences_annuelles_{annee1}"]

    # fusion avec la géométrie
    gdf_plot = gdf_dep.merge(df_merge, left_on="code", right_on="code_dep", how="left")

    # plot
    fig, ax = plt.subplots(figsize=(10, 12))

    gdf_plot.plot(
        column="taux",
        ax=ax,
        cmap="coolwarm",   # bleu = négatif, rouge = positif
        legend=True,
        edgecolor="grey",
        linewidth=0.5,
        vmin=-0.5,
        vmax=0.5,
        missing_kwds={
            "color": "lightgrey",
            "edgecolor": "grey",
            "hatch": "//",
            "label": "Données manquantes"
        }
    )

    ax.set_title(
        title if title else f"Taux de croissance ({'tous les sports' if sport=='all' else sport}) des licenciés ({annee1} - {annee2})",
        fontsize=16
    )
    ax.set_axis_off()
    plt.show()


In [ ]:
plot_evolution(2023, 2024, 'TDT')

In [ ]:
# GeoDataFrame des départements
gdf = gpd.read_file("./departements.geojson")  # adapte le chemin si nécessaire
gdf["code"] = gdf["code"].astype(str)  # s'assurer que c'est string

# On remplace les NaN dans Code_sport par "Unknown"
data_licences_clean["code_sport"] = data_licences_clean["code_sport"].fillna("Unknown")

# Création des menus déroulants
annee = sorted(data_licences_clean["annee"].unique())
codes_sports = sorted(data_licences_clean["code_sport"].unique())

annee_widget = widgets.Dropdown(options=["all"] + annee, description="Année :", value="all")
sport_widget = widgets.Dropdown(options=["all"] + codes_sports, description="Sport:", value="all")

# Fonction pour filtrer et agréger les licences
def filter_aggregate(df, year, sport):
    df_filtered = df.copy()
    if year != "all":
        df_filtered = df_filtered[df_filtered["annee"] == year]
    if sport != "all":
        df_filtered = df_filtered[df_filtered["code_sport"] == sport]
    return df_filtered.groupby("code_dep")["licences_annuelles"].sum().reset_index()

# Fonction pour tracer la carte
def plot_map(df_agg, title="Licences par département"):
    # Merge avec le GeoDataFrame
    gdf_plot = gdf.merge(df_agg, left_on="code", right_on="code_dep", how="left")
    gdf_plot["licences_annuelles"] = gdf_plot["licences_annuelles"].fillna(0)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    gdf_plot.plot(column="licences_annuelles", cmap="OrRd", linewidth=0.5, edgecolor="gray",
                  legend=True, ax=ax, norm=Normalize(vmin=0, vmax=gdf_plot["licences_annuelles"].max()))
    ax.set_title(title, fontsize=16)
    ax.axis("off")
    plt.show()

# Callback pour mettre à jour les cartes
def update_maps(change=None):
    clear_output(wait=True)
    display(annee_widget, sport_widget)
    
    df1 = filter_aggregate(data_licences_clean, annee_widget.value, sport_widget.value)
    
    plot_map(
        df1,
        title=f"Nombre de licenciés ({'tous les sports' if sport_widget.value=='all' else sport_widget.value}) "
              f"par département ({'2016-2024' if annee_widget.value=='all' else annee_widget.value})"
    )

# Liaison des widgets à la fonction de callback
annee_widget.observe(update_maps, names='value')
sport_widget.observe(update_maps, names='value')

# Affichage initial
display(annee_widget, sport_widget)
update_maps()


In [ ]:
# GeoDataFrame des départements
gdf = gpd.read_file("./departements.geojson")
gdf["code"] = gdf["code"].astype(str)

# Remplacer les NaN dans Code_sport
data_licences_clean["code_sport"] = data_licences_clean["code_sport"].fillna("Unknown")

# Menus déroulants
annees = sorted(data_licences_clean["annee"].unique())
codes_sports = sorted(data_licences_clean["code_sport"].unique())

annee1_widget = widgets.Dropdown(options=annees, description="Année 1:", value=annees[0])
annee2_widget = widgets.Dropdown(options=annees, description="Année 2:", value=annees[-1])
sport_widget = widgets.Dropdown(options=["all"] + codes_sports, description="Sport:", value="all")

# Filtrer et agréger
def filter_aggregate(df, year, sport):
    df_filtered = df.copy()
    if sport != "all":
        df_filtered = df_filtered[df_filtered["code_sport"] == sport]
    df_year = df_filtered[df_filtered["annee"] == year]
    return df_year.groupby("code_dep")["licences_annuelles"].sum().reset_index()

# Carte évolution en %
def plot_evolution_map(year1, year2, sport):
    df1 = filter_aggregate(data_licences_clean, year1, sport)
    df2 = filter_aggregate(data_licences_clean, year2, sport)
    
    # Merge pour calculer delta
    df_merge = df1.merge(df2, on="code_dep", how="outer", suffixes=("_y1", "_y2")).fillna(0)
    df_merge["pct_change"] = ((df_merge["licences_annuelles_y2"] - df_merge["licences_annuelles_y1"]) /
                              df_merge["licences_annuelles_y1"].replace({0: 1})) * 100  # éviter div0
    
    # Merge avec GeoDataFrame
    gdf_plot = gdf.merge(df_merge, left_on="code", right_on="code_dep", how="left")
    gdf_plot["pct_change"] = gdf_plot["pct_change"].fillna(0)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    norm = TwoSlopeNorm(vmin=gdf_plot["pct_change"].min(),
                        vcenter=0,
                        vmax=gdf_plot["pct_change"].max())
    
    gdf_plot.plot(column="pct_change", cmap="coolwarm", linewidth=0.5, edgecolor="gray",
                  legend=True, norm=norm, ax=ax)
    
    ax.set_title(f"Évolution des licenciés {'tous les sports' if sport_widget.value=='all' else sport_widget.value} par département ({year1} - {year2})")
    ax.axis("off")
    plt.show()

# Callback
def update_map(change=None):
    clear_output(wait=True)
    display(annee1_widget, annee2_widget, sport_widget)
    plot_evolution_map(annee1_widget.value, annee2_widget.value, sport_widget.value)

# Liaison widgets
annee1_widget.observe(update_map, names='value')
annee2_widget.observe(update_map, names='value')
sport_widget.observe(update_map, names='value')

# Affichage initial
display(annee1_widget, annee2_widget, sport_widget)
update_map()
